In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import confusion_matrix

In [89]:
images = pd.read_csv('hw02_images.csv',header=None)
labels = pd.read_csv('hw02_labels.csv',header=None)

In [90]:
train_x = images.iloc[:30000,:]
test_x = images.iloc[30000:35000]
train_y = labels.iloc[0:30000]
test_y = labels.iloc[30000:35000]

In [91]:
print(len(train_x))

30000


In [92]:
tshirt_df = train_x[train_y[0]==1]
dress_df = train_x[train_y[0]==2]
coat_df = train_x[train_y[0]==3]
shirt_df = train_x[train_y[0]==4]
bag_df = train_x[train_y[0]==5]

In [46]:
means =  np.array([tshirt_df.mean(), dress_df.mean(), coat_df.mean(), shirt_df.mean(), bag_df.mean()])

print("(means[:,0]) \n", means)

(means[:,0]) 
 [[254.99696707 254.98266898 254.83188908 ... 254.82019064 254.944974
  254.97616984]
 [254.99294533 254.99294533 254.99074074 ... 254.98765432 254.99559083
  254.99514991]
 [255.         255.         254.99726651 ... 251.87471526 254.64829157
  254.98132118]
 [254.99252419 254.99032542 254.92348285 ... 251.78540018 254.27264732
  254.87730871]
 [254.99865832 254.99463327 254.99194991 ... 250.53243848 253.18076063
  254.7950783 ]]


In [93]:
deviations = np.array([np.sqrt(np.mean(((tshirt_df-means[0])**2))), 
                       np.sqrt(np.mean(((dress_df-means[1]))**2)), 
                       np.sqrt(np.mean(((coat_df-means[2]))**2)), 
                       np.sqrt(np.mean(((shirt_df-means[3]))**2)),
                       np.sqrt(np.mean(((bag_df-means[4]))**2))])

print("(deviations[:,0]) \n", deviations)

(deviations[:,0]) 
 [[ 0.09129318  0.25609545  1.31113235 ...  5.30014721  3.91230365
   1.93966398]
 [ 0.2065885   0.2065885   0.21645843 ...  1.04093685  0.47058513
   0.70065226]
 [ 0.05163978  0.04082483  0.16010067 ... 18.43997833  6.79044523
   1.10615342]
 [ 0.18440729  0.21617172  1.81049333 ... 15.68985315  6.34774223
   1.80069771]
 [ 0.04471148  0.64590555  3.03296839 ... 23.62618241 13.9167999
   4.47278072]]


In [94]:
train_priors = (len(train_y[train_y[0]==1]) / len(train_y)), (len(train_y[train_y[0]==2]) / len(train_y)), (len(train_y[train_y[0]==3]) / len(train_y)), (len(train_y[train_y[0]==4]) / len(train_y)), (len(train_y[train_y[0]==5]) / len(train_y))

print("(priors[:,0]) \n", train_priors)

(priors[:,0]) 
 (0.2, 0.2, 0.2, 0.2, 0.2)


In [95]:
def posterior_prob(x):
    label1 = sum((-0.5*np.log(2*math.pi)) - (np.log(deviations[0])) - ((x-means[0])*(x-means[0])/(2*deviations[0]*deviations[0]))) + 0.2 
    label2 = sum((-0.5*np.log(2*math.pi)) - (np.log(deviations[1])) - ((x-means[1])*(x-means[1])/(2*deviations[1]*deviations[1]))) + 0.2
    label3 = sum((-0.5*np.log(2*math.pi)) - (np.log(deviations[2])) - ((x-means[2])*(x-means[2])/(2*deviations[2]*deviations[2]))) + 0.2
    label4 = sum((-0.5*np.log(2*math.pi)) - (np.log(deviations[3])) - ((x-means[3])*(x-means[3])/(2*deviations[3]*deviations[3]))) + 0.2
    label5 = sum((-0.5*np.log(2*math.pi)) - (np.log(deviations[4])) - ((x-means[4])*(x-means[4])/(2*deviations[4]*deviations[4]))) + 0.2

    lst = []
    lst.append(label1)
    lst.append(label2)
    lst.append(label3)
    lst.append(label4)
    lst.append(label5)
    max_value = max(lst)
    max_index = lst.index(max_value)

    return max_index

In [100]:
np.array(confusion_matrix(train_x.apply(lambda x: posterior_prob(x), axis = 1), train_y))

array([[   0, 3683,   49,    3,  678,    5],
       [   0, 1436, 5668, 1137, 1379,  528],
       [   0,  500,  207, 4675, 2953,  902],
       [   0,  239,   60,  123,  687,  178],
       [   0,  142,   16,   62,  303, 4387],
       [   0,    0,    0,    0,    0,    0]])

In [99]:
np.array(confusion_matrix(test_x.apply(lambda x: posterior_prob(x), axis = 1), test_y))

array([[  0, 599,   6,   0, 112,   1],
       [  0, 237, 955, 188, 266,  81],
       [  0,  90,  25, 786, 465, 168],
       [  0,  34,  11,  15, 109,  28],
       [  0,  40,   3,  11,  48, 722],
       [  0,   0,   0,   0,   0,   0]])